### Carregando funções Turing

In [1]:
%run -i '/home/hadoop/.Turing/TuringAllFunctions.py'

------------- Turing Lab Data Science --------------------
------ Funções carregadas em memória com sucesso ---------
--------- Data da última atualização: --------------------
--------------- 13/05/2019 -------------------------------
----------------------------------------------------------
--- Consulte o manual de Biblioteca de Funções Turing: ---


----------------------------------------------------------


### Parêmetros do Projeto

In [2]:
#---------- Definição dos Parâmetros do projeto ---------------------------------
nm_bucket = 'turing-bkt-treinamentos'
id_empresa = '1022'

# Id do projeto parametrizado no Turing (o mesmo valor dado ao campo Id Projeto do Timeline)
id_projeto = '350002'

# Nome da tabela utilizada para desenvolver o modelo
nm_file_desenv_orig = 'ABT01_TITANIC_DESENV.csv'

# Nome da tabela utilizada para validar o modelo (out-of-time ou out-of-sample)
nm_file_valida_orig = 'OOS01_TITANIC_VALID.csv'

# Nome da variável resposta (quando houve)
targetname = 'Survived'

# Delimitador da tabela 
abt_delimiter = ','

 

### Credenciais de acessos ao projeto

In [5]:
################### Não alterar este parágrafo #####################################
#---------- Gerando Credenciais para Acesso Seguro ---------------------------------
id_modelo = id_projeto
schema = 'EMP_'+ id_empresa +'_Proj_'+ id_projeto
S3fs,S3session,S3client,S3resource = TuringUsersCredentialsControl(id_empresa)
schema_rds = 'EMP_'+id_empresa+'_Proj_'+id_projeto
mysq_conn,engine = TuringRDSManagementConn(id_empresa,schema_rds,turing_adm = 'N')
path_s3_wfile = 's3://'+nm_bucket+'/Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'

# Caminho do projeto
path_proj = 'Projetos/'+'Projeto_'+id_projeto+'/Modelo_Id_'+id_projeto+'/Modeltrain/'
s3_path = path_s3_wfile
####################################################################################

#### Lendo tabela de input para Turing

In [12]:
# Lendo tabela de desenvolvimento original do bucket da POC
path_file = 'Projetos/FLAT_TABLES/'+str(nm_file_desenv_orig)
desenv_df = TuringReadS3CSVFile(nm_bucket,path_file,sep=abt_delimiter)
desenv_df.shape

(623, 12)

In [13]:
desenv_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,858,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S,1
1,53,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C,1
2,387,3,"Goodwin, Master. Sidney Leonard",male,1.0,5,2,CA 2144,46.9000,NaN,S,0
3,125,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D26,S,0
4,579,3,"Caram, Mrs. Joseph (Maria Elias)",female,NaN,1,0,2689,14.4583,NaN,C,0


#### Lendo tabela de desenvolvimento do modelo

In [21]:
nm_tab_int = 'abt_desenv.csv'
abt_desenv = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
abt_desenv.rename( columns={'Unnamed: 0':'id'}, inplace=True )
abt_desenv.shape

(485, 7)

In [22]:
abt_desenv.head()

,id,Fare,Age,Pclass,TGD_Sex_female,TGD_Sex_male,target_t_bin
0,11,-0.516566,0.000000,0.848448,1,0,0
1,496,-0.196153,0.313981,-0.342054,1,0,1
2,468,0.498508,0.000000,0.848448,0,1,0
3,424,-0.512419,0.000000,0.848448,1,0,1
4,576,-0.455387,0.467718,-0.342054,0,1,0


#### Lendo tabela de Validação

In [23]:
# Lendo tabela de desenvolvimento original do bucket da POC
nm_tab_int = 'abt_valida.csv'
abt_valid = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
abt_valid.rename( columns={'Unnamed: 0':'id'}, inplace=True )
abt_valid.shape

(138, 7)

In [24]:
abt_valid.head()

,id,Fare,Age,Pclass,TGD_Sex_female,TGD_Sex_male,target_t_bin
0,313,-0.339250,1.082665,0.848448,0,1,0
1,387,-0.326807,-0.454703,0.848448,1,0,1
2,214,-0.403540,0.313981,-0.342054,1,0,1
3,60,-0.283515,-1.453991,0.848448,0,1,0
4,343,-0.417020,0.006508,-0.342054,1,0,1


#### Parâmetris do Grid Search - busca por melhores hiperparâmetros

In [25]:
# Lendo tabela de desenvolvimento original do bucket da POC
nm_tab_int = 'grid_search_param.csv'
grid_search_param = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
grid_search_param.shape

(10, 71)

In [27]:
grid_search_param.head(100)

,Unnamed: 0,C,class_weight,dual,fit_intercept,max_iter,modelo,multi_class,n_jobs,penalty,...,min_child_weight,missing,nthread,objective,reg_alpha,reg_lambda,scale_pos_weight,seed,silent,intercept_scaling
0,0,1.00,None,False,True,100.0,Regressao Logistica,ovr,1.0,l2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.01,None,NaN,NaN,-1.0,Support Vector Classification,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,None,NaN,True,100.0,Perceptron,NaN,1.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,None,NaN,NaN,NaN,Random Forest Classifier,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,None,NaN,NaN,NaN,Decision Tree Classifier,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,NaN,NaN,NaN,NaN,NaN,k-nearest neighbors vote,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,NaN,None,NaN,True,100.0,Linear Classifiers with SGD,NaN,1.0,l2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,NaN,NaN,NaN,NaN,NaN,Gradient Boosting,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,XGBoost,NaN,1.0,NaN,...,1.0,None,None,binary:logistic,0.0,1.0,1.0,None,True,NaN
9,0,1.00,None,True,True,1000.0,Linear SVC,ovr,NaN,l2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
#### Lendo tabela de metadados completa - gerada pelo Turing

In [28]:
# Lendo tabela de desenvolvimento original do bucket da POC
nm_tab_int = 'db_metadados_full.csv'
db_metadados_full = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
db_metadados_full.shape

(12, 22)

In [29]:
db_metadados_full

,Unnamed: 0,index,Features,Role,Level,Keep,Tipo,Cardinality,qt_missing,pc_miss,...,unique,top,freq,mean,std,min,25%,50%,75%,max
0,0,0,PassengerId,id,ordinal,1,int64,623,0,0,...,NaN,NaN,NaN,447.678973,258.248011,1.00,217.500,449.0,670.5000,890.0000
1,1,1,Pclass,input,ordinal,1,int64,3,0,0,...,NaN,NaN,NaN,2.287319,0.840657,1.00,1.500,3.0,3.0000,3.0000
2,2,2,Name,input,nominal,1,object,623,0,0,...,623.0,"Rugg, Miss. Emily",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,Sex,input,nominal,1,object,2,0,0,...,2.0,male,408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,Age,input,interval,1,float64,83,121,19,...,NaN,NaN,NaN,29.915339,14.506994,0.67,21.000,29.0,38.0000,80.0000
5,5,5,SibSp,input,ordinal,1,int64,7,0,0,...,NaN,NaN,NaN,0.531300,1.162891,0.00,0.000,0.0,1.0000,8.0000
6,6,6,Parch,input,ordinal,1,int64,7,0,0,...,NaN,NaN,NaN,0.393258,0.832085,0.00,0.000,0.0,0.0000,6.0000
7,7,7,Ticket,input,nominal,1,object,510,0,0,...,510.0,CA. 2343,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8,Fare,input,interval,1,float64,216,0,0,...,NaN,NaN,NaN,32.458273,48.257658,0.00,7.925,15.0,31.3875,512.3292
9,9,9,Cabin,input,nominal,1,object,120,471,75,...,120.0,C23 C25 C27,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Lendo tabela com as variaveis do modelo

In [30]:
# Lendo tabela de desenvolvimento original do bucket da POC
nm_tab_int = 'turing_variable_modelo.csv'
turing_variable_modelo = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
turing_variable_modelo.shape

(5, 3)

In [31]:
turing_variable_modelo

,Unnamed: 0,Variaveis_Modelo,Importancia
0,0,Fare,0.159
1,1,Age,0.159
2,2,Pclass,0.122
3,3,TGD_Sex_female,0.236
4,4,TGD_Sex_male,0.212


#### Lendo tabela com lista de variáveis excluídas por quantidade de missing

In [32]:
# Lendo tabela de desenvolvimento original do bucket da POC
nm_tab_int = 'lista_vars_missing_rejected.csv'
lista_vars_missing_rejected = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
lista_vars_missing_rejected.shape

(0, 22)

In [33]:
lista_vars_missing_rejected

,Unnamed: 0,index,Features,Role,Level,Keep,Tipo,Cardinality,qt_missing,pc_miss,...,unique,top,freq,mean,std,min,25%,50%,75%,max


#### Lendo tabela original de entrada

In [40]:
path_file = path_proj+'train_df_full.parquet'
train_df_full = TuringReadParquetFile(S3fs,nm_bucket,path_file)
train_df_full.shape

(623, 12)

In [41]:
train_df_full.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,858,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S,1
1,53,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C,1
2,387,3,"Goodwin, Master. Sidney Leonard",male,1.0,5,2,CA 2144,46.9000,None,S,0
3,125,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D26,S,0
4,579,3,"Caram, Mrs. Joseph (Maria Elias)",female,NaN,1,0,2689,14.4583,None,C,0


#### Lendo tabela pós data prep

In [42]:
path_file = path_proj+'turing_dataprep.parquet'
turing_dataprep = TuringReadParquetFile(S3fs,nm_bucket,path_file)
turing_dataprep.shape

(623, 12)

In [37]:
turing_dataprep.head()

,Fare,Age,SibSp,Pclass,Parch,TGD_Sex_female,TGD_Sex_male,TGD_Embarked_C,TGD_Embarked_Q,TGD_Embarked_S,Survived,WoE_REJECT
0,-0.122530,1.620744,-0.457246,-1.532556,-0.472998,0,1,0,0,1,1,0
1,0.918124,1.467007,0.403371,-1.532556,-0.472998,1,0,1,0,0,1,0
2,0.299503,-2.222675,3.845840,0.848448,1.932533,0,1,0,0,1,0,0
3,0.929702,1.851349,-0.457246,-1.532556,0.729768,0,1,0,0,1,0,0
4,-0.373297,0.000000,0.403371,0.848448,-0.472998,1,0,1,0,0,0,0


#### Lendo tabela com quadro comparativo dos modelos treinados

In [43]:
# Lendo tabela de desenvolvimento original do bucket da POC
nm_tab_int = 'model_comparison.csv'
model_comparison = TuringReadS3CSVFile(nm_bucket,path_proj+nm_tab_int,sep=abt_delimiter)
model_comparison.shape

(11, 9)

In [45]:
model_comparison[['Model','Gini','AUCP','AUCR','Tempo_sec']]

,Model,Gini,AUCP,AUCR,Tempo_sec
0,Logistic Regression,0.8228,0.8741,0.9114,0.02
1,Linear SVC,0.8228,0.8740,0.9114,0.03
2,Stochastic Gradient Descent,0.8228,0.8780,0.9114,0.02
3,Support Vector Machines,0.8220,0.8772,0.9110,0.25
4,Random Forest,0.8058,0.8799,0.9029,5.67
5,Naive Bayes,0.7998,0.8363,0.8999,0.01
6,XGBoost Classifier,0.7986,0.8792,0.8993,1.36
7,Gradient Boosting Classifier,0.7970,0.8099,0.8985,3.05
8,Decision Tree,0.7564,0.8668,0.8782,0.19
9,KNN,0.7430,0.8420,0.8715,0.03
